# Generate results notebook

In this notebook the model is simulated and the results are generated. 

1. First, the necessary libraries and the model name are imported. 
2. The uncertainties are defined and they are given an uncertainty range.
    Moreover, the model outcomes are provided. 
3. Lastly, the results are saved

## 1.  Import libraries \& data

In [1]:
from warnings import filterwarnings
filterwarnings(action='ignore', category=DeprecationWarning, message='`np.bool` is a deprecated alias')
from ema_workbench import ( CategoricalParameter, perform_experiments, RealParameter,  ema_logging, \
                           TimeSeriesOutcome, Constant, save_results, load_results, MultiprocessingEvaluator )
from ema_workbench.connectors.vensim import VensimModel

C:\Users\User\Anaconda3\lib\site-packages\ema_workbench\connectors\__init__.py:32: ImportWarning: simio connector not available
  warnings.warn("simio connector not available", ImportWarning)


In [2]:
from __future__ import (absolute_import, print_function, division, unicode_literals)
from ema_workbench import (Model, RealParameter, CategoricalParameter, ScalarOutcome, Constant,\
                           Policy, perform_experiments, ema_logging, \
                           TimeSeriesOutcome, perform_experiments,save_results, load_results)
#from ema_workbench.em_framework.evaluators import LHS, SOBOL, MORRIS
from ema_workbench.analysis import (clusterer, plotting, Density, pairs_plotting, get_ex_feature_scores, \
                                    RuleInductionType, feature_scoring)
from ema_workbench.analysis import scenario_discovery_util as sdutil
from ema_workbench.analysis.plotting_util import do_titles,do_ylabels, TIME
import ema_workbench.analysis.plotting_util as plt_util
from ema_workbench.analysis.plotting import group_by_envelopes, single_envelope, plot_lines_with_envelopes, prepare_data 
from ema_workbench.analysis.plotting import simple_kde, group_density, plot_envelope, simple_density
from ema_workbench.analysis.plotting import lines, envelopes, kde_over_time, multiple_densities
from ema_workbench.analysis.pairs_plotting import pairs_scatter, pairs_density
import ema_workbench.analysis.cart as cart
from ema_workbench.analysis.pairs_plotting import (pairs_lines, pairs_scatter,pairs_density)
import numpy as np
import seaborn as sns #; sns.set(style="ticks", color_codes=True)
import pandas as pd
import matplotlib.pyplot as plt
TIME_LABEL = 'Time'

import importlib
importlib.reload(sdutil)
importlib.reload(cart)

import itertools
import datetime
import math
import matplotlib.gridspec as gridspec
import scipy.stats.kde as kde
from matplotlib.colors import ColorConverter
from matplotlib.collections import PolyCollection, PathCollection
import matplotlib.pyplot as plt
from matplotlib.pyplot import pie
from matplotlib.ticker import FormatStrFormatter, FuncFormatter
from matplotlib.patches import ConnectionPatch
import matplotlib.font_manager as fm
import matplotlib as mpl

C:\Users\User\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. I

In [6]:
labels_time = [2000, 2010, 2020, 2030, 2040, 2050]
plt.rcParams['axes.xmargin'] = 0
plt.rcParams['axes.ymargin'] = 0
plt.rcParams['legend.frameon'] = False

In [7]:
wd = 'C:/Users/User/Documents/Studie/EPA/Master Thesis/Simulation'
model = VensimModel('Vensim', wd = wd, model_file= 'model_ndfeb_subst.vpmx')

## 2. Define uncertainties \& model outcomes

In [8]:
model.uncertainties = [
    # Gonna include
    RealParameter("Uncertainty factor weight NdFeB in application", 0.7, 1.3),
    RealParameter("Uncertainty factor disassembly rate NdFeB", 0.7 , 1.3 ),
    RealParameter("Fraction extra production costs for a lower magnet content", 0.2, 0.5),
    RealParameter("Decay factor", 0.01 , 0.05 ),    
    RealParameter("Uncertainty factor marginal cost", 0.5 , 2 ),
    RealParameter("Uncertainty factor demand growth rate", 0.7 , 1.3 ),
    RealParameter('"Low coercive magnet-for-magnet substitution threshold"', 1.5, 3 ),
    RealParameter('"High coercive magnet-for-magnet substitution threshold"', 2, 4 ),
    RealParameter("Uncertainty factor share offshore wind", 0.7, 1.3),
    
    RealParameter("Price elasticity short term", 0.02 , 0.08 ),
    RealParameter("Price elasticity long term", 0.1 , 0.2 ),
    RealParameter("Maximum increase production capacity", 0.1 , 0.2 ),
    RealParameter("Maximum decrease production capacity", 0.01 , 0.05 ),
    RealParameter("Proposed mine lifetime", 20 , 30 ),
    RealParameter("Smelter and refiner usage investment cap", 0.6 , 0.9 ),
    RealParameter("Minimum stock transit time", 0.05 , 0.3 ),
    RealParameter("Uncertainty factor change in production capacity", 0.7 , 1.3 ),
    RealParameter("Minimum price of illegal mining", 1 , 4 ),
    RealParameter("Substitution amplifying factor", 0.8 , 1.2 ),
    
    RealParameter("Long term substitution strength", 0.05 , 0.2 ),
    RealParameter("Short term substitution strength", 0.01 , 0.03 ),    
    RealParameter("Uncertainty factor lifetime", 0.7, 1.3),
    
    CategoricalParameter("Switch SSP", (1,2,3,4)),
    CategoricalParameter("Switch energy price growth scenario", (1,2,3)),
    CategoricalParameter("Switch technology readiness HTS", (0,1)),
    CategoricalParameter("Switch technology readiness SRM", (0,1)),
    CategoricalParameter("Switch technology readiness ASM high", (0,1)),
    CategoricalParameter("Switch annual newly installed wind capacity", (1,2,3,4)),
    CategoricalParameter("Switch vehicle scenarios", (0,1)),
    CategoricalParameter("Switch disturbance", (0,1,2,3)),
]

In [9]:
model.outcomes = [
    TimeSeriesOutcome("Time"),

    # Cars
    TimeSeriesOutcome("Fraction newly produced motor type per car type[BEV,PSM]"),
    TimeSeriesOutcome("Fraction newly produced motor type per car type[BEV,ASM]"),
    TimeSeriesOutcome("Fraction newly produced motor type per car type[BEV,EESM]"),
    TimeSeriesOutcome("Fraction newly produced motor type per car type[BEV,SRM]"),
    TimeSeriesOutcome("Fraction newly produced motor type per car type[BEV,ASM high]"),
    TimeSeriesOutcome("Fraction newly produced motor type per car type[BEV,Hybrid]"),

    TimeSeriesOutcome("Fraction newly produced motor type per car type[HEV,PSM]"),
    TimeSeriesOutcome("Fraction newly produced motor type per car type[HEV,ASM]"),
    TimeSeriesOutcome("Fraction newly produced motor type per car type[HEV,EESM]"),
    TimeSeriesOutcome("Fraction newly produced motor type per car type[HEV,SRM]"),
    TimeSeriesOutcome("Fraction newly produced motor type per car type[HEV,ASM high]"),
    TimeSeriesOutcome("Fraction newly produced motor type per car type[HEV,Hybrid]"),

    TimeSeriesOutcome("Fraction newly produced motor type per car type[PHEV,PSM]"),
    TimeSeriesOutcome("Fraction newly produced motor type per car type[PHEV,ASM]"),
    TimeSeriesOutcome("Fraction newly produced motor type per car type[PHEV,EESM]"),
    TimeSeriesOutcome("Fraction newly produced motor type per car type[PHEV,SRM]"),
    TimeSeriesOutcome("Fraction newly produced motor type per car type[PHEV,ASM high]"),
    TimeSeriesOutcome("Fraction newly produced motor type per car type[PHEV,Hybrid]"),
    
    TimeSeriesOutcome("Fraction of newly produced cars with PM per car type[PHEV]"),
    TimeSeriesOutcome("Fraction of newly produced cars with PM per car type[HEV]"),
    TimeSeriesOutcome("Fraction of newly produced cars with PM per car type[BEV]"),
    
    TimeSeriesOutcome("Fraction of newly produced cars with PM"),
    
    TimeSeriesOutcome("Fraction motor type per car type in use[BEV,PSM]"),
    TimeSeriesOutcome("Fraction motor type per car type in use[BEV,ASM]"),
    TimeSeriesOutcome("Fraction motor type per car type in use[BEV,EESM]"),
    TimeSeriesOutcome("Fraction motor type per car type in use[BEV,SRM]"),
    TimeSeriesOutcome("Fraction motor type per car type in use[BEV,ASM high]"),
    TimeSeriesOutcome("Fraction motor type per car type in use[BEV,Hybrid]"),

    TimeSeriesOutcome("Fraction motor type per car type in use[HEV,PSM]"),
    TimeSeriesOutcome("Fraction motor type per car type in use[HEV,ASM]"),
    TimeSeriesOutcome("Fraction motor type per car type in use[HEV,EESM]"),
    TimeSeriesOutcome("Fraction motor type per car type in use[HEV,SRM]"),
    TimeSeriesOutcome("Fraction motor type per car type in use[HEV,ASM high]"),
    TimeSeriesOutcome("Fraction motor type per car type in use[HEV,Hybrid]"),

    TimeSeriesOutcome("Fraction motor type per car type in use[PHEV,PSM]"),
    TimeSeriesOutcome("Fraction motor type per car type in use[PHEV,ASM]"),
    TimeSeriesOutcome("Fraction motor type per car type in use[PHEV,EESM]"),
    TimeSeriesOutcome("Fraction motor type per car type in use[PHEV,SRM]"),
    TimeSeriesOutcome("Fraction motor type per car type in use[PHEV,ASM high]"),
    TimeSeriesOutcome("Fraction motor type per car type in use[PHEV,Hybrid]"),
    
    TimeSeriesOutcome("Fraction of cars in use with PM per car type[PHEV]"),
    TimeSeriesOutcome("Fraction of cars in use with PM per car type[HEV]"),
    TimeSeriesOutcome("Fraction of cars in use with PM per car type[BEV]"),
    
    TimeSeriesOutcome("Fraction of cars in use with PM"),

    TimeSeriesOutcome("Newly produced cars[BEV,PSM]"),
    TimeSeriesOutcome("Newly produced cars[BEV,ASM]"),
    TimeSeriesOutcome("Newly produced cars[BEV,EESM]"),
    TimeSeriesOutcome("Newly produced cars[BEV,SRM]"),
    TimeSeriesOutcome("Newly produced cars[BEV,ASM high]"),
    TimeSeriesOutcome("Newly produced cars[BEV,Hybrid]"),

    TimeSeriesOutcome("Newly produced cars[HEV,PSM]"),
    TimeSeriesOutcome("Newly produced cars[HEV,ASM]"),
    TimeSeriesOutcome("Newly produced cars[HEV,EESM]"),
    TimeSeriesOutcome("Newly produced cars[HEV,SRM]"),
    TimeSeriesOutcome("Newly produced cars[HEV,ASM high]"),
    TimeSeriesOutcome("Newly produced cars[HEV,Hybrid]"),

    TimeSeriesOutcome("Newly produced cars[PHEV,PSM]"),
    TimeSeriesOutcome("Newly produced cars[PHEV,ASM]"),
    TimeSeriesOutcome("Newly produced cars[PHEV,EESM]"),
    TimeSeriesOutcome("Newly produced cars[PHEV,SRM]"),
    TimeSeriesOutcome("Newly produced cars[PHEV,ASM high]"),
    TimeSeriesOutcome("Newly produced cars[PHEV,Hybrid]"),

    TimeSeriesOutcome("Cars in use[BEV,PSM]"),
    TimeSeriesOutcome("Cars in use[BEV,ASM]"),
    TimeSeriesOutcome("Cars in use[BEV,EESM]"),
    TimeSeriesOutcome("Cars in use[BEV,SRM]"),
    TimeSeriesOutcome("Cars in use[BEV,ASM high]"),
    TimeSeriesOutcome("Cars in use[BEV,Hybrid]"),

    TimeSeriesOutcome("Cars in use[HEV,PSM]"),
    TimeSeriesOutcome("Cars in use[HEV,ASM]"),
    TimeSeriesOutcome("Cars in use[HEV,EESM]"),
    TimeSeriesOutcome("Cars in use[HEV,SRM]"),
    TimeSeriesOutcome("Cars in use[HEV,ASM high]"),
    TimeSeriesOutcome("Cars in use[HEV,Hybrid]"),

    TimeSeriesOutcome("Cars in use[PHEV,PSM]"),
    TimeSeriesOutcome("Cars in use[PHEV,ASM]"),
    TimeSeriesOutcome("Cars in use[PHEV,EESM]"),
    TimeSeriesOutcome("Cars in use[PHEV,SRM]"),
    TimeSeriesOutcome("Cars in use[PHEV,ASM high]"),
    TimeSeriesOutcome("Cars in use[PHEV,Hybrid]"),

    TimeSeriesOutcome("Cars in use total per car type[PHEV]"),
    TimeSeriesOutcome("Cars in use total per car type[HEV]"),
    TimeSeriesOutcome("Cars in use total per car type[BEV]"),

    TimeSeriesOutcome("Total cars in use"),
    
    TimeSeriesOutcome("Factor availability of REE cars"),

    TimeSeriesOutcome("Demand REE from cars[Neodymium]"),
    TimeSeriesOutcome("Demand REE from cars[Praseodymium]"),
    TimeSeriesOutcome("Demand REE from cars[Dysprosium]"),
    TimeSeriesOutcome("Demand REE from cars[Terbium]"),

    TimeSeriesOutcome("Production capacity cars[BEV,PSM]"),
    TimeSeriesOutcome("Production capacity cars[BEV,ASM]"),
    TimeSeriesOutcome("Production capacity cars[BEV,EESM]"),
    TimeSeriesOutcome("Production capacity cars[BEV,SRM]"),
    TimeSeriesOutcome("Production capacity cars[BEV,ASM high]"),
    TimeSeriesOutcome("Production capacity cars[BEV,Hybrid]"),    

    TimeSeriesOutcome("Production capacity cars[HEV,PSM]"),
    TimeSeriesOutcome("Production capacity cars[HEV,ASM]"),
    TimeSeriesOutcome("Production capacity cars[HEV,EESM]"),
    TimeSeriesOutcome("Production capacity cars[HEV,SRM]"),
    TimeSeriesOutcome("Production capacity cars[HEV,ASM high]"),
    TimeSeriesOutcome("Production capacity cars[HEV,Hybrid]"),

    TimeSeriesOutcome("Production capacity cars[PHEV,PSM]"),
    TimeSeriesOutcome("Production capacity cars[PHEV,ASM]"),
    TimeSeriesOutcome("Production capacity cars[PHEV,EESM]"),
    TimeSeriesOutcome("Production capacity cars[PHEV,SRM]"),
    TimeSeriesOutcome("Production capacity cars[PHEV,ASM high]"),
    TimeSeriesOutcome("Production capacity cars[PHEV,Hybrid]"), 

    # Wind turbines
    TimeSeriesOutcome("Fraction wind capacity installation per turbine type per wind location[Onshore, PMG geared]"),
    TimeSeriesOutcome("Fraction wind capacity installation per turbine type per wind location[Onshore, PMG DD]"),
    TimeSeriesOutcome("Fraction wind capacity installation per turbine type per wind location[Onshore, DFIG]"),
    TimeSeriesOutcome("Fraction wind capacity installation per turbine type per wind location[Onshore, SCIG]"),
    TimeSeriesOutcome("Fraction wind capacity installation per turbine type per wind location[Onshore, HTS]"),
    TimeSeriesOutcome("Fraction wind capacity installation per turbine type per wind location[Onshore, EESG]"),

    TimeSeriesOutcome("Fraction wind capacity installation per turbine type per wind location[Offshore, PMG geared]"),
    TimeSeriesOutcome("Fraction wind capacity installation per turbine type per wind location[Offshore, PMG DD]"),
    TimeSeriesOutcome("Fraction wind capacity installation per turbine type per wind location[Offshore, DFIG]"),
    TimeSeriesOutcome("Fraction wind capacity installation per turbine type per wind location[Offshore, SCIG]"),
    TimeSeriesOutcome("Fraction wind capacity installation per turbine type per wind location[Offshore, HTS]"),
    TimeSeriesOutcome("Fraction wind capacity installation per turbine type per wind location[Offshore, EESG]"),
    
    TimeSeriesOutcome("Fraction wind capacity installation with NdFeB per wind location[Onshore]"),
    TimeSeriesOutcome("Fraction wind capacity installation with NdFeB per wind location[Offshore]"),
    
    TimeSeriesOutcome("Fraction wind capacity installation with NdFeB"),

    TimeSeriesOutcome("Fraction installed wind capacity per turbine type per wind location[Onshore, PMG geared]"),
    TimeSeriesOutcome("Fraction installed wind capacity per turbine type per wind location[Onshore, PMG DD]"),
    TimeSeriesOutcome("Fraction installed wind capacity per turbine type per wind location[Onshore, DFIG]"),
    TimeSeriesOutcome("Fraction installed wind capacity per turbine type per wind location[Onshore, SCIG]"),
    TimeSeriesOutcome("Fraction installed wind capacity per turbine type per wind location[Onshore, HTS]"),
    TimeSeriesOutcome("Fraction installed wind capacity per turbine type per wind location[Onshore, EESG]"),

    TimeSeriesOutcome("Fraction installed wind capacity per turbine type per wind location[Offshore, PMG geared]"),
    TimeSeriesOutcome("Fraction installed wind capacity per turbine type per wind location[Offshore, PMG DD]"),
    TimeSeriesOutcome("Fraction installed wind capacity per turbine type per wind location[Offshore, DFIG]"),
    TimeSeriesOutcome("Fraction installed wind capacity per turbine type per wind location[Offshore, SCIG]"),
    TimeSeriesOutcome("Fraction installed wind capacity per turbine type per wind location[Offshore, HTS]"),
    TimeSeriesOutcome("Fraction installed wind capacity per turbine type per wind location[Offshore, EESG]"),
    
    TimeSeriesOutcome("Fraction installed wind capacity with NdFeB per wind location[Onshore]"),
    TimeSeriesOutcome("Fraction installed wind capacity with NdFeB per wind location[Offshore]"),
    
    TimeSeriesOutcome("Fraction installed wind capacity with NdFeB"),

    TimeSeriesOutcome("Wind capacity installation[Onshore, PMG geared]"),
    TimeSeriesOutcome("Wind capacity installation[Onshore, PMG DD]"),
    TimeSeriesOutcome("Wind capacity installation[Onshore, DFIG]"),
    TimeSeriesOutcome("Wind capacity installation[Onshore, SCIG]"),
    TimeSeriesOutcome("Wind capacity installation[Onshore, HTS]"),
    TimeSeriesOutcome("Wind capacity installation[Onshore, EESG]"),

    TimeSeriesOutcome("Wind capacity installation[Offshore, PMG geared]"),
    TimeSeriesOutcome("Wind capacity installation[Offshore, PMG DD]"),
    TimeSeriesOutcome("Wind capacity installation[Offshore, DFIG]"),
    TimeSeriesOutcome("Wind capacity installation[Offshore, SCIG]"),
    TimeSeriesOutcome("Wind capacity installation[Offshore, HTS]"),
    TimeSeriesOutcome("Wind capacity installation[Offshore, EESG]"),

    TimeSeriesOutcome("Wind capacity in use[Onshore, PMG geared]"),
    TimeSeriesOutcome("Wind capacity in use[Onshore, PMG DD]"),
    TimeSeriesOutcome("Wind capacity in use[Onshore, DFIG]"),
    TimeSeriesOutcome("Wind capacity in use[Onshore, SCIG]"),
    TimeSeriesOutcome("Wind capacity in use[Onshore, HTS]"),
    TimeSeriesOutcome("Wind capacity in use[Onshore, EESG]"),

    TimeSeriesOutcome("Wind capacity in use[Offshore, PMG geared]"),
    TimeSeriesOutcome("Wind capacity in use[Offshore, PMG DD]"),
    TimeSeriesOutcome("Wind capacity in use[Offshore, DFIG]"),
    TimeSeriesOutcome("Wind capacity in use[Offshore, SCIG]"),
    TimeSeriesOutcome("Wind capacity in use[Offshore, HTS]"),
    TimeSeriesOutcome("Wind capacity in use[Offshore, EESG]"),

    TimeSeriesOutcome("Wind capacity per wind location[Onshore]"),
    TimeSeriesOutcome("Wind capacity per wind location[Offshore]"),

    TimeSeriesOutcome("Total wind capacity in use"),

    TimeSeriesOutcome("Demand REE from wind industry[Neodymium]"),
    TimeSeriesOutcome("Demand REE from wind industry[Praseodymium]"),
    TimeSeriesOutcome("Demand REE from wind industry[Dysprosium]"),
    TimeSeriesOutcome("Demand REE from wind industry[Terbium]"),

    TimeSeriesOutcome("Production capacity WTG[Onshore,PMG geared]"),
    TimeSeriesOutcome("Production capacity WTG[Onshore,PMG DD]"),
    TimeSeriesOutcome("Production capacity WTG[Onshore,DFIG]"),
    TimeSeriesOutcome("Production capacity WTG[Onshore,SCIG]"),
    TimeSeriesOutcome("Production capacity WTG[Onshore,HTS]"),
    TimeSeriesOutcome("Production capacity WTG[Onshore,EESG]"),        

    TimeSeriesOutcome("Production capacity WTG[Offshore,PMG geared]"),
    TimeSeriesOutcome("Production capacity WTG[Offshore,PMG DD]"),
    TimeSeriesOutcome("Production capacity WTG[Offshore,DFIG]"),
    TimeSeriesOutcome("Production capacity WTG[Offshore,SCIG]"),
    TimeSeriesOutcome("Production capacity WTG[Offshore,HTS]"),
    TimeSeriesOutcome("Production capacity WTG[Offshore,EESG]"),
    
    TimeSeriesOutcome("Factor availability REE for WT"),

    # Demand
    TimeSeriesOutcome("Postponed demand[Neodymium]"),    
    TimeSeriesOutcome("Postponed demand[Praseodymium]"), 
    TimeSeriesOutcome("Postponed demand[Terbium]"), 
    TimeSeriesOutcome("Postponed demand[Dysprosium]"),

    TimeSeriesOutcome("Total demand REE[Neodymium]"),    
    TimeSeriesOutcome("Total demand REE[Praseodymium]"), 
    TimeSeriesOutcome("Total demand REE[Terbium]"), 
    TimeSeriesOutcome("Total demand REE[Dysprosium]"),     

    # Price
    TimeSeriesOutcome("Real price REE[Neodymium]"),    
    TimeSeriesOutcome("Real price REE[Praseodymium]"), 
    TimeSeriesOutcome("Real price REE[Terbium]"), 
    TimeSeriesOutcome("Real price REE[Dysprosium]"), 

    TimeSeriesOutcome("Material composition NdFeB magnet low coercivity[Neodymium]"),    
    TimeSeriesOutcome("Material composition NdFeB magnet low coercivity[Praseodymium]"), 
    TimeSeriesOutcome("Material composition NdFeB magnet low coercivity[Terbium]"), 
    TimeSeriesOutcome("Material composition NdFeB magnet low coercivity[Dysprosium]"),

    TimeSeriesOutcome("Price NdFeB high coercivity"),
    TimeSeriesOutcome("Price NdFeB magnet low coercivity"),
    
    TimeSeriesOutcome("LREE and HREE content NdFeB magnet low coercivity[Neodymium]"),
    TimeSeriesOutcome("LREE and HREE content NdFeB magnet low coercivity[Dysprosium]"),
    
    # Supply chain
    TimeSeriesOutcome("Industrial ore mining[Neodymium]"),
    TimeSeriesOutcome("Industrial ore mining[Praseodymium]"), 
    TimeSeriesOutcome("Industrial ore mining[Terbium]"), 
    TimeSeriesOutcome("Industrial ore mining[Dysprosium]"),

    TimeSeriesOutcome("Illegal ore trade[Neodymium]"),    
    TimeSeriesOutcome("Illegal ore trade[Praseodymium]"), 
    TimeSeriesOutcome("Illegal ore trade[Terbium]"), 
    TimeSeriesOutcome("Illegal ore trade[Dysprosium]"),

    TimeSeriesOutcome("Part illegal mining of primary production[Neodymium]"),
    TimeSeriesOutcome("Part illegal mining of primary production[Praseodymium]"),
    TimeSeriesOutcome("Part illegal mining of primary production[Terbium]"),
    TimeSeriesOutcome("Part illegal mining of primary production[Dysprosium]"),

     # Industrial and illegal mining capacity
    TimeSeriesOutcome("Industrial mining capacity[Mineral]"),    
    TimeSeriesOutcome("Industrial mining capacity[Ion adsorped clay]"),

    TimeSeriesOutcome("Industrial mining capacity REE[Neodymium]"),    
    TimeSeriesOutcome("Industrial mining capacity REE[Praseodymium]"), 
    TimeSeriesOutcome("Industrial mining capacity REE[Terbium]"), 
    TimeSeriesOutcome("Industrial mining capacity REE[Dysprosium]"),

    TimeSeriesOutcome("Illegal mining capacity[Neodymium]"),    
    TimeSeriesOutcome("Illegal mining capacity[Praseodymium]"), 
    TimeSeriesOutcome("Illegal mining capacity[Terbium]"), 
    TimeSeriesOutcome("Illegal mining capacity[Dysprosium]"),
    
    TimeSeriesOutcome("Parking of industrial mining capacity[Mineral]"),    
    TimeSeriesOutcome("Parking of industrial mining capacity[Ion adsorped clay]"),
    
    TimeSeriesOutcome("Decomissioning of industrial mines due to age[Mineral]"),    
    TimeSeriesOutcome("Decomissioning of industrial mines due to age[Ion adsorped clay]"),
    
    TimeSeriesOutcome("Preparation of industrial capacity increase[Mineral]"),    
    TimeSeriesOutcome("Preparation of industrial capacity increase[Ion adsorped clay]"),
    
    TimeSeriesOutcome("Preparation of industrial capacity increase[Mineral]"),    
    TimeSeriesOutcome("Preparation of industrial capacity increase[Ion adsorped clay]"),

    # Demand other applications
    TimeSeriesOutcome("Magnet for magnet substitution per REE[Neodymium]"), 
    TimeSeriesOutcome("Magnet for magnet substitution per REE[Praseodymium]"),  
    TimeSeriesOutcome("Magnet for magnet substitution per REE[Dysprosium]"),  
    TimeSeriesOutcome("Magnet for magnet substitution per REE[Terbium]"),  
    TimeSeriesOutcome("Magnet for magnet substitution"), 
    
    TimeSeriesOutcome("Substitution REE other applications[Electric bus,Neodymium]"),  
    TimeSeriesOutcome("Substitution REE other applications[Electric bus,Praseodymium]"),  
    TimeSeriesOutcome("Substitution REE other applications[Electric bus,Terbium]"),  
    TimeSeriesOutcome("Substitution REE other applications[Electric bus,Dysprosium]"),  

    TimeSeriesOutcome("Substitution REE other applications[Electric truck,Neodymium]"),  
    TimeSeriesOutcome("Substitution REE other applications[Electric truck,Praseodymium]"),  
    TimeSeriesOutcome("Substitution REE other applications[Electric truck,Terbium]"),  
    TimeSeriesOutcome("Substitution REE other applications[Electric truck,Dysprosium]"),  

    TimeSeriesOutcome("Demand REE other applications[Electric bus,Neodymium]"),  
    TimeSeriesOutcome("Demand REE other applications[Electric bus,Praseodymium]"),  
    TimeSeriesOutcome("Demand REE other applications[Electric bus,Terbium]"),  
    TimeSeriesOutcome("Demand REE other applications[Electric bus,Dysprosium]"),  

    TimeSeriesOutcome("Demand REE other applications[Electric truck,Neodymium]"),  
    TimeSeriesOutcome("Demand REE other applications[Electric truck,Praseodymium]"),  
    TimeSeriesOutcome("Demand REE other applications[Electric truck,Terbium]"),  
    TimeSeriesOutcome("Demand REE other applications[Electric truck,Dysprosium]"),  

    # Demand EV ad wind
    TimeSeriesOutcome("Demand EV and wind[Onshore]"),  
    TimeSeriesOutcome("Demand EV and wind[Offshore]"),  
    TimeSeriesOutcome("Demand EV and wind[HEV]"),  
    TimeSeriesOutcome("Demand EV and wind[BEV]"),    
    TimeSeriesOutcome("Demand EV and wind[PHEV]"),

]

In [10]:
#model.constants = [ Constant("Switch disturbance", 0) ]

## 3. Save the results

In [11]:
results = perform_experiments(model,5000)

100%|████████████████████████████████████| 5000/5000 [2:34:47<00:00,  1.86s/it]


In [12]:
experiments, outcomes = results

In [13]:
save_results(results, 'Results/Model_runs_wo_cluster.tar.gz')

C:\Users\User\Anaconda3\lib\site-packages\ema_workbench\em_framework\outcomes.py:515: UserWarning: still to be tested!!
  warnings.warn("still to be tested!!")
